AUTHOR: [@sparshsah](github.com/sparshsah)

# BOND STATS

I propose the following approximation: `irr` $\approx$ `coupon - loss - premium / wal`, where
* `irr` is internal rate of return (annualized);
* `coupon` is coupon fraction (annualized);
* `loss` is (gross-of-recoveries) loss rate (annualized);
* `premium` is the premium (if positive) or discount (if negative) paid, in percentage points; and
* `wal` is weighted-average life.

For simplicity, I neglect recoveries, which makes net-of-recoveries loss rate the same as `loss`. WLOG, I assume initial principal is \$1 and payments are made at the end of each year (this is WLOG because instead of an Earth year, you can make the unit of time anything you want, and call that your year).

I'd like to know how good this approximation is under various
* Amortization schedules
* Coupons
* Loss rates
* Premia
* Terms.

In [1]:
from typing import Final

import numpy as np
import pandas as pd

In [2]:
PRIN: Final[float] = 1.00

In [80]:
def _get_pmts(
    amort: bool = False,
    term: int = 1,
    loss_rate: float = 0.00,
    coupon_frac: float = 0.00,
    prin: float = PRIN,
    px: float = 1.00,
) -> pd.DataFrame:
    """DF(index=timesteps, columns=["prin", "chargeoff", "coupon"])."""
    pmts = pd.DataFrame(
        index=range(0, term+1),
        columns=["prin", "chargeoff", "coupon"],
    )
    # special row
    pmts.loc[0, :] = pd.Series({"prin": -px*prin, "chargeoff": 0, "coupon": 0})
    bal = prin
    for t in range(1, term+1):
        pmts.loc[t, "coupon"] = bal * coupon_frac
        pmts.loc[t, "chargeoff"] = bal * loss_rate
        bal -= pmts.loc[t, "chargeoff"]
        ####
        if amort:
            # beginning-of-period remaining term -- fencepost principle
            rterm = term - t + 1
            pmts.loc[t, "prin"] = bal / rterm
        else:
            pmts.loc[t, "prin"] = 0 if t < term else bal
        bal -= pmts.loc[t, "prin"]       
    assert np.isclose(bal, 0), f"Balance should be zero, is {bal}!"
    # augment column
    pmts.loc[:, "net"] = pmts.sum(axis="columns")
    return pmts

In [81]:
_get_pmts(
    amort=False,
    term=6,
    loss_rate=0.10,
    coupon_frac=0.20,
)

,prin,chargeoff,coupon,net
0,-1.0,0.0,0.0,-1.000000
1,0,0.1,0.2,0.300000
2,0,0.09,0.18,0.270000
3,0,0.081,0.162,0.243000
4,0,0.0729,0.1458,0.218700
5,0,0.06561,0.13122,0.196830
6,0.531441,0.059049,0.118098,0.708588
